# Notebook error analysis 

## Refining error types - Clustering Value errors


use OPTICS with GloVe on value errors


## 1. nbs from GitHub

### 1.1 Load cluster result
Already clustered during the method selection process, now just load the result and choose the best one (optics with word2vec/glove)

In [19]:
# read in all errors from the file
import pandas as pd
import numpy as np 
import cluster_util

path = r"C:\Users\yirwa29\Downloads\Dataset-Nb\nbdata_g_error\analysis_gerr"
path_file = path+"/cluster_selection_g_valueerror/nberror_g_cluster_valueerror.xlsx"

df2_err_valueerr_clusters = pd.read_excel(path_file)
df2_err_valueerr_clusters = df2_err_valueerr_clusters[['fname','evalue','evalue_tokenized','traceback','lib_parsed','cluster_optics_glove']]
df2_err_valueerr_clusters.head()

,fname,evalue,evalue_tokenized,traceback,lib_parsed,cluster_optics_glove
0,00000-1017-heatmapseq2seq.ipynb,"x and y must have same first dimension, but ha...",x and y must have same first dimension but hav...,['--------------------------------------------...,matplotlib,0
1,00000-1572-corona-tweet-sentiment-analysis.ipynb,in user code:\n\n /home/arun/.local/lib/pyt...,in user code return self iterator outputs args...,['--------------------------------------------...,tensorflow,-1
2,00000-1580-unet-of-cnn-airfoil.ipynb,"x and y must have same first dimension, but ha...",x and y must have same first dimension but hav...,['--------------------------------------------...,matplotlib,0
3,00000-1588-9-hackerearth-ml-all-classifiers-en...,Classification metrics can't handle a mix of c...,classification metrics ca nt handle a mix of c...,['--------------------------------------------...,sklearn,37
4,00000-1711-ls-ds-433-autoencoders-lecture.ipynb,"num must be 1 <= num <= 20, not 0",num must be num not,['--------------------------------------------...,matplotlib,75


In [20]:
path_value_err = path+'/cluster_g_valueerror'
df2_err_valueerr_clusters.to_excel(path_value_err+'/cluster_res.xlsx', index=False, engine='xlsxwriter')

### 1.2 sort
Sort the clustering result of the value errors, manually inspect the clustered ones, and recluster the noise samples.

In [28]:
# save sorted clusters
df2_err_valueerr_clusters[df2_err_valueerr_clusters['cluster_optics_glove']!=-1].sort_values('cluster_optics_glove').to_excel(path_value_err+'/cluster_res_sorted.xlsx', index=False, engine='xlsxwriter')
# save only noise samples
df2_err_valueerr_clusters[df2_err_valueerr_clusters['cluster_optics_glove']==-1].sort_values('cluster_optics_glove').to_excel(path_value_err+'/cluster_res_noise.xlsx', index=False, engine='xlsxwriter')

In [29]:
# clustered points
len(df2_err_valueerr_clusters.loc[df2_err_valueerr_clusters['cluster_optics_glove']!=-1])

8502

In [30]:
# noise points
len(df2_err_valueerr_clusters.loc[df2_err_valueerr_clusters['cluster_optics_glove']==-1])

7068

Load all the unclustered samples (clustered as noise) of the value errors

try to cluster them again

In [1]:
import pandas as pd
import config
import numpy as np
import cluster_util

path = r"C:\Users\yirwa29\Downloads\Dataset-Nb\nbdata_g_error\analysis_gerr"
path_value_err = path+'/cluster_g_valueerror'
df2_err_valueerr_clusters_noise = pd.read_excel(path_value_err+'/cluster_res_noise.xlsx')

# # GloVe embeddings
# glove_vectors = cluster_util.load_glove("C:/Users/yirwa29/Downloads/Dataset-Nb/glove.6B/glove.6B.200d.txt")
# X2_noise_glove = np.array([cluster_util.vectorizer_word2vec(xi, glove_vectors, 200) for xi in df2_err_valueerr_clusters_noise['evalue_tokenized'].values.astype('U')])

# with open(path_value_err+'/nberror_g_valueerror_noise_embeddings_glove.txt', 'wb') as f:
#     np.save(f, X2_noise_glove)

with open(path_value_err+'/nberror_g_valueerror_noise_embeddings_glove.txt', 'rb') as f:
    X2_noise_glove = np.load(f)

In [47]:
# total amount of noise samples
len(df2_err_valueerr_clusters_noise)

7068

In [2]:
# see how many noise samples are ML library related
len(df2_err_valueerr_clusters_noise[df2_err_valueerr_clusters_noise.lib_parsed.isin(config.top_lib_names)])

4953

try to lower the min samples - still too many noise points

In [5]:
res2_noise = cluster_util.cluster_optics(X2_noise_glove, min_samples = 5)

Estimated no. of clusters: 363
Estimated no. of noise points: 3489


C:\Users\yirwa29\AppData\Local\anaconda3\envs\chatgpt\Lib\site-packages\sklearn\cluster\_optics.py:1081: RuntimeWarning: divide by zero encountered in divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


maybe try PCA to reduce dimension

GloVe dimension=200 -> 50?20?10?2?

In [83]:
X2_noise_glove_rd = cluster_util.pca(X2_noise_glove, n_component = 2)

In [84]:
# this is so far the best, but still many noise
res2_noise = cluster_util.cluster_optics(X2_noise_glove_rd, min_samples = 5)

Estimated no. of clusters: 533
Estimated no. of noise points: 1617


C:\Users\yirwa29\AppData\Local\anaconda3\envs\chatgpt\Lib\site-packages\sklearn\cluster\_optics.py:1081: RuntimeWarning: divide by zero encountered in divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


In [85]:
# cluster score
cluster_util.eval_cluster_silhouette(X2_noise_glove_rd[res2_noise!=-1], res2_noise[res2_noise!=-1])

0.7452030664694608

In [86]:
df2_err_valueerr_clusters_noise["cluster_optics_glove"]=res2_noise
len(df2_err_valueerr_clusters_noise[df2_err_valueerr_clusters_noise.cluster_optics_glove==-1 & df2_err_valueerr_clusters_noise.lib_parsed.isin(config.top_lib_names)])

6

In [88]:
df2_err_valueerr_clusters_noise.to_excel(path_value_err+'/cluster_res_noise_reclustered.xlsx', index=False, engine='xlsxwriter')

In [65]:
eps2 = cluster_util.epsilon_search_dbscan(X2_noise_glove_rd)
print(eps2)
res2_noise2 = cluster_util.cluster_dbscan(X2_noise_glove_rd, eps=eps2, min_samples=5)

0.6316662692131638
Estimated no. of clusters: 96
Estimated no. of noise points: 792


In [66]:
# cluster score        negative...???
cluster_util.eval_cluster_silhouette(X2_noise_glove_rd[res2_noise2!=-1], res2_noise2[res2_noise2!=-1])

-0.08378285822404327

In [67]:
X2_noise_transformers = cluster_util.vectorizer_sentence2vec(df2_err_valueerr_clusters_noise['evalue_tokenized'].values.astype('U'))
res2_noise = cluster_util.cluster_optics(X2_noise_transformers, min_samples = 5) # still too many noise points
# cluster score
cluster_util.eval_cluster_silhouette(X2_noise_transformers[res2_noise!=-1], res2_noise[res2_noise!=-1])

C:\Users\yirwa29\AppData\Local\anaconda3\envs\chatgpt\Lib\site-packages\sklearn\cluster\_optics.py:1081: RuntimeWarning: divide by zero encountered in divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


Estimated no. of clusters: 381
Estimated no. of noise points: 3245


0.79985183

In [68]:
X2_noise_transformers_rd = cluster_util.pca(X2_noise_transformers, n_component = 10)

In [69]:
res2_noise = cluster_util.cluster_optics(X2_noise_transformers_rd, min_samples = 5) # still too many noise points

Estimated no. of clusters: 461
Estimated no. of noise points: 3387


C:\Users\yirwa29\AppData\Local\anaconda3\envs\chatgpt\Lib\site-packages\sklearn\cluster\_optics.py:1081: RuntimeWarning: divide by zero encountered in divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


In [70]:
# cluster score
cluster_util.eval_cluster_silhouette(X2_noise_transformers_rd[res2_noise!=-1], res2_noise[res2_noise!=-1])

0.7622353

In [71]:
eps2 = cluster_util.epsilon_search_dbscan(X2_noise_transformers_rd)
print(eps2)
res2_noise2 = cluster_util.cluster_dbscan(X2_noise_transformers_rd, eps=eps2, min_samples=5)
# cluster score
cluster_util.eval_cluster_silhouette(X2_noise_transformers_rd[res2_noise2!=-1], res2_noise2[res2_noise2!=-1])

0.18483255172777158
Estimated no. of clusters: 248
Estimated no. of noise points: 1871


0.36284482

## 2. nbs from Kaggle

### 2.1 perform clustering

In [ ]:
import pandas as pd
import numpy as np 
import cluster_util

path = r"C:\Users\yirwa29\Downloads\Dataset-Nb\nbdata_k_error\analysis_kerr/cluster_k_valueerror"
path_file = path+"/nberror_k_cluster_valueerror.xlsx"
# read in all errors from the file
df_err = pd.read_excel(r"C:\Users\yirwa29\Downloads\Dataset-Nb\nbdata_k_error/nberror_k_p5_evaluetokenized.xlsx")
# get only valueerror
df_err_ename = df_err[df_err['ename']=='valueerror']
err_descs = df_err_ename['evalue_tokenized'].values.astype('U')

# GloVe embeddings
glove_vectors = cluster_util.load_glove("C:/Users/yirwa29/Downloads/Dataset-Nb/glove.6B/glove.6B.200d.txt")
X_glove = np.array([cluster_util.vectorizer_word2vec(xi, glove_vectors, 200) for xi in err_descs])
print("The shape of the GloVe embedding is", X_glove.shape)
# save embeddings
with open(path+'/nberror_k_valueerror_embeddings_glove.txt', 'wb') as f:
    np.save(f, X_glove)

# save a checkpoint
df_err_ename.to_excel(path_file, index=False, engine='xlsxwriter')

In [1]:
# load a checkpoint
import pandas as pd
import numpy as np 
import cluster_util

path = r"C:\Users\yirwa29\Downloads\Dataset-Nb\nbdata_k_error\analysis_kerr/cluster_k_valueerror"
path_file = path+"/nberror_k_cluster_valueerror.xlsx"
df_err_ename = pd.read_excel(path_file)
with open(path+'/nberror_k_valueerror_embeddings_glove.txt', 'rb') as f:
    X_glove = np.load(f)

C:\Users\yirwa29\AppData\Local\anaconda3\envs\chatgpt\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# optics clustering 
res = cluster_util.cluster_optics(X_glove, min_samples = 5)
df_err_ename.loc[:,"cluster_optics_glove"] = res

Estimated no. of clusters: 58
Estimated no. of noise points: 370


C:\Users\yirwa29\AppData\Local\anaconda3\envs\chatgpt\Lib\site-packages\sklearn\cluster\_optics.py:1081: RuntimeWarning: divide by zero encountered in divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


In [10]:
# save the cluster result
df_err_ename.to_excel(path_file, index=False, engine='xlsxwriter')

### 2.2 sort

In [14]:
import pandas as pd
import numpy as np 
import cluster_util

path = r"C:\Users\yirwa29\Downloads\Dataset-Nb\nbdata_k_error\analysis_kerr/cluster_k_valueerror"
path_file = path+"/nberror_k_cluster_valueerror.xlsx"
df_err_ename = pd.read_excel(path_file)

# save sorted clusters
df_err_ename.loc[df_err_ename['cluster_optics_glove']!=-1].sort_values('cluster_optics_glove').to_excel(path+'/cluster_res_sorted.xlsx', index=False, engine='xlsxwriter')
# save only noise samples
df_err_ename.loc[df_err_ename['cluster_optics_glove']==-1].sort_values('cluster_optics_glove').to_excel(path+'/cluster_res_noise.xlsx', index=False, engine='xlsxwriter')

recluster the noise samples

In [16]:
import pandas as pd

path = r"C:\Users\yirwa29\Downloads\Dataset-Nb\nbdata_k_error\analysis_kerr/cluster_k_valueerror"
df_err_ename = pd.read_excel(path+'/cluster_res_noise.xlsx')
df_err_ename

,fname,ename,evalue,evalue_tokenized,traceback,lib_parsed,cluster_optics_glove
0,ahmadfadliramadhan_covid-19-south-korea-visual...,valueerror,attempt to get argmax of an empty sequence,attempt to get argmax of an empty sequence,['--------------------------------------------...,pandas,-1
1,vaniatsai_bimbo-0000.ipynb,valueerror,Input data must be 2 dimensional and non empty.,input data must be dimensional and non empty,['--------------------------------------------...,numpy,-1
2,tanbeerjubaer_first-notebook.ipynb,valueerror,Cannot assign value to variable ' embedding_2/...,can not assign value to variable shape mismatc...,['--------------------------------------------...,tensorflow,-1
3,tanbeerjubaer_first-notebook.ipynb,valueerror,dtype='numeric' is not compatible with arrays ...,dtypenumeric is not compatible with arrays of ...,['--------------------------------------------...,sklearn,-1
4,tanbeerjubaer_first-notebook.ipynb,valueerror,setting an array element with a sequence. The ...,setting an array element with a sequence the r...,['--------------------------------------------...,sklearn,-1
...,...,...,...,...,...,...,...
365,akshit8459_hiring-candidate.ipynb,valueerror,Type of input is not string! Please enter a va...,type of input is not string please enter a val...,['--------------------------------------------...,pandas,-1
366,yiqwantang_item-demand-sarima.ipynb,valueerror,The indices for endog and exog are not aligned,the indices for endog and exog are not aligned,['--------------------------------------------...,statsmodels,-1
367,yfwei36_small-businesses-loans-analysis.ipynb,valueerror,could not convert string to float: 'ABC HOBBYC...,could not convert string to float abc hobbycraft,['--------------------------------------------...,pandas,-1
368,dhairyapandya1_duplicate-question-quora.ipynb,valueerror,feature_names must be unique,must be unique,['--------------------------------------------...,xgboost,-1
